In [1]:
import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

#time
from datetime import datetime
from datetime import timedelta

import jieba
import jieba.analyse
jieba.set_dictionary('dict.idkrsi.txt')     # 改預設字典
jieba.analyse.set_stop_words("stopword.goatwang.kang.txt") #指定stopwords字典

# get data
# ! conda install pandas-datareader s
#import pandas_datareader as pdr

# visual
# ! pip install mpl-finance
#import matplotlib.pyplot as plt
#import mpl_finance as mpf

#import seaborn as sns


# https://github.com/mrjbq7/ta-lib
# ! pip install ta-lib
#import talib

In [2]:
df_bbs = pd.read_csv("bda2019_dataset/bbs2.csv",encoding="utf-8")
df_forum = pd.read_csv("bda2019_dataset/forum2.csv",encoding="utf-8")
df_news = pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
df_news['comment_count']=0

df_article = pd.concat([df_forum, df_bbs, df_news])  #三個合併
del df_bbs, df_forum, df_news

df_article['post_time'] = pd.to_datetime(df_article['post_time'])
df_article['post_time2'] = df_article['post_time'].dt.date   # .dt.date用在dataframe  .date()用在一個 #只留日期
#df_article['label'] = 'even'
df_article['content'] = df_article['content'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article['title'] = df_article['title'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article = df_article.sort_values(by=['post_time']).reset_index(drop=True) # 用post_time排序 # 在重設index
df_article.head(2)
#df_article2 = df_article[['post_time2','title','content']]

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
0,鉅亨網,0,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,1451580107540_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 00:06:00,重大要聞,yahoo股市,淪落的2015 竟是78年來最難獲利的一年,2016-01-01
1,Bokolo,10,1.原文連結(必須檢附)： http://magazine.chinatimes.com/m...,1451580710262_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1451580509.A.722...,2016-01-01 00:48:25,Stock,Ptt,[新聞] 第三方支付 台灣普及關鍵年,2016-01-01


In [3]:
df_TWSE2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv",encoding="utf-8")
df_TWSE2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv",encoding="utf-8")
df_TWSE2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv",encoding="utf-8")

df_TWSE = pd.concat([df_TWSE2016, df_TWSE2017, df_TWSE2018])  #三年合併
del df_TWSE2016, df_TWSE2017, df_TWSE2018

# ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '成交值(千元)', '成交筆數(筆)', '流通在外股數(千股)', '本益比-TSE', '股價淨值比-TSE']
df_TWSE['證券代碼'] = df_TWSE['證券代碼'].astype(str)
df_TWSE['年月日'] = pd.to_datetime(df_TWSE['年月日'])
df_TWSE['開盤價(元)'] = df_TWSE['開盤價(元)'].str.replace(',' , '').astype('float64') # 1,000 to 1000 to float
df_TWSE['最高價(元)'] = df_TWSE['最高價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['最低價(元)'] = df_TWSE['最低價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['收盤價(元)'] = df_TWSE['收盤價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交量(千股)'] = df_TWSE['成交量(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['成交值(千元)'] = df_TWSE['成交值(千元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交筆數(筆)'] = df_TWSE['成交筆數(筆)'].str.replace(',' , '').astype('int64')
df_TWSE['流通在外股數(千股)'] = df_TWSE['流通在外股數(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['本益比-TSE'] = df_TWSE['本益比-TSE'].str.replace(',' , '').astype('float64')
df_TWSE['股價淨值比-TSE'] = df_TWSE['股價淨值比-TSE'].astype('float64')
df_TWSE.head(2) 

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016-12-30,29.09,29.64,28.93,29.64,6121.0,213121.0,2939,3692175.0,20.56,1.23
1,1102 亞泥,2016-12-30,24.91,24.91,24.67,24.81,3347.0,88078.0,1227,3361447.0,31.75,0.73


In [4]:
# 選那家股票
#company_name = '國巨'
company_name = '奇力新'


# 文章包含那家字
#company_words = '被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|凱美|同欣電|大毅|君耀|普斯|國巨'
company_words = '被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|飛磁|旺詮|美磊|美桀|向華科技|奇力新'

# 漲跌幾%
PA = 0.05

# even幾%
PAE = 0.003

In [5]:
# 用日期排序 再把index重排
#2327 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True)
#2456 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('奇力新')].sort_values(by=['年月日']).reset_index(drop=True)
#2478 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('大毅')].sort_values(by=['年月日']).reset_index(drop=True)
#6271 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('同欣電')].sort_values(by=['年月日']).reset_index(drop=True)

df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains(company_name)].sort_values(by=['年月日']).reset_index(drop=True)
del df_TWSE
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46


In [6]:
##增欄位:fluctuation幅度 tag漲跌平
df_trend['fluctuation'] = 0.0
df_trend['tag']='--'
df_trend['closeshift'] = 0.0
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,0.0,--,0.0
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.0,--,0.0


In [7]:
# ##增欄位:fluctuation幅度 tag漲跌平
# df_trend['fluctuation'] = 0.0
# df_trend['tag']='--'

# ###計算漲跌
# for index, row in df_trend.iterrows():
#     try:
#         margin =(float(df_trend.loc[index,'收盤價(元)']) - float(df_trend.loc[index-1,'收盤價(元)']) )/ float(df_trend.loc[index-1,'收盤價(元)'])
#         df_trend.loc[index,'fluctuation']=margin
#         if margin >=0.03:
#             df_trend.loc[index,'tag']='up'
#         elif margin <= -0.03:
#             df_trend.loc[index,'tag']='down'
#         else:
#             df_trend.loc[index,'tag']='even'
#     except:
#         continue

In [8]:
df_trend['closeshift'] = df_trend['收盤價(元)'].shift(periods=1)#.fillna(value=0.0, inplace=True)
#df_trend['closeshift'].fillna(value= 0.0, inplace=True)
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,0.0,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.0,--,80.2


In [9]:
df_trend['fluctuation'] = (df_trend['收盤價(元)'] - df_trend['closeshift']) / df_trend['closeshift']
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,NaN,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.060599,--,80.2


In [10]:
df_trend.loc[df_trend['fluctuation'] >=  PA, 'tag'] = 'up'
df_trend.loc[df_trend['fluctuation'] <= -PA, 'tag'] = 'down'
df_trend.loc[(df_trend['fluctuation'] >= -PAE) & (df_trend['fluctuation'] <= PAE), 'tag'] = 'even'
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,NaN,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.060599,up,80.2


In [11]:
len(df_trend[df_trend['tag']=='up'])

37

In [12]:
len(df_trend[df_trend['tag']=='down'])

39

In [13]:
len(df_trend[df_trend['tag']=='even'])

90

In [14]:
#df_company = df_article[ df_article['content'].str.contains('國巨')]   # df 某欄位 string contains "國巨"
#df_company = df_article[ df_article['content'].str.contains('奇力新')] 
#df_company = df_article[ df_article['content'].str.contains('大毅')]  
#df_company = df_article[ df_article['content'].str.contains('同欣電 ')]
df_company = df_article[ df_article['content'].str.contains(company_words)]   
print(len(df_company))
del df_article
df_company.head(2)

10553


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,2016-01-01
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,2016-01-02


In [15]:
df_company.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,2016-01-01
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,2016-01-02


In [16]:
df_trend.loc[2,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[1,'年月日'].date()

True

In [17]:
df_trend.loc[5,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[4,'年月日'].date()

False

In [18]:
d = df_trend.loc[1,'年月日'].date() - df_trend.loc[ 1-1 ,'年月日'].date() #相減差幾天
d

datetime.timedelta(1)

In [19]:
d.days #只取天數

1

In [20]:
int(d.days) #幾天 轉整數

1

In [21]:
df_trend.loc[3,'年月日'].date() 

datetime.date(2016, 1, 7)

In [22]:
df_company[ df_company['post_time2'] == df_trend.loc[3,'年月日'].date() ].head() # 某欄位 == n 的 全部撈出來

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
999,Moneydj理財網,0,1.週三台股在美股重挫以及陸股再度啟動融斷機制之下，指數重挫點，指數跌破十年線，所幸尾盤大型...,1452156767683_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 16:29:00,重大要聞,yahoo股市,日盛投顧：台股短線有反彈機會,2016-01-07


In [23]:
# # 演算法 
# for index, row in df_2327.iterrows():
#     try:
#         if df_2327.loc[index,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[index-1,'年月日'].date():                        
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差1天  # 那前1天的文章標上當天的漲跌   
        
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-2) == df_2327.loc[index-1,'年月日'].date():           
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差2天 #那前2天的文章標上當天的漲跌
            
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-3) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-4) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-4), 'label'] = df_2327.loc[index,'tag']


#     except:
#         continue

In [24]:
# 看所有相差的天數
# for index, row in df_2327.iterrows():
#     try:
#         n = df_2327.loc[index,'年月日'].date() - df_2327.loc[index-1,'年月日'].date()
#         print(n)
#     except:
#         continue

# 最多12天

In [25]:
# 如果股票前一筆差n天  # 那前n天的文章標上當天的漲跌 
df_company['label5566']='--'
for index, row in df_trend.iterrows():
    try:
        n = int((df_trend.loc[index,'年月日'].date() - df_trend.loc[index-1,'年月日'].date()).days ) # 差幾個datetime # 轉天數 # 再轉整數
#        print(n)

        for i in range(1, n+1):
#            print(i)
            df_company.loc[df_company['post_time2'] ==  df_trend.loc[index,'年月日'].date() +  timedelta(days=-i), 'label5566'] = df_trend.loc[index,'tag']
    except:
        continue

In [26]:
print(len(df_company[df_company['label5566']=='down']))
df_company[df_company['label5566']=='down'].head(2)

1284


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,label5566
1522,財訊快報,0,◆全台勞檢啟動，11類行業小心，其中284家電子零組件企業，實施全面普查。<BR> ◆海外分...,1452472319234_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:05:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-11,down
1530,時報資訊,0,陸股暫停熔斷機制，減緩市場對於系統性風險的疑慮，法人指出，台股短線壓力仍大，若結構出現惡化現...,1452472329341_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:15:00,重大要聞,yahoo股市,《基金》全球經濟趨緩，慎選台股標的,2016-01-11,down


In [27]:
print(len(df_company[df_company['label5566']=='up']))
df_company[df_company['label5566']=='up'].head(2)

891


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,label5566
222,財訊快報,0,◆千億科技預算，張善政喊加碼，主計總處初步匡列1 061億、年增3%，可能再調高。<BR> ...,1451867210012_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 07:56:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-04,up
289,時報資訊,0,2016年開紅盤日，台股狂洩逾200點！儘管2015年最後一個交易日拉尾盤，守住8300點，...,1451879808470_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 11:34:00,重大要聞,yahoo股市,《盤中解析》紅盤日變臉，台股狂洩逾200點,2016-01-04,up


In [28]:
print(len(df_company[df_company['label5566']=='even']))
df_company[df_company['label5566']=='even'].head(2)

911


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,label5566
3121,鉅亨網,0,受到月線反壓、外資籌碼面持續偏保守的影響下，台股期現貨今(20)日同步下壓，加權指數開低走低...,1453258680867_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-20 10:42:00,重大要聞,yahoo股市,台股盤中─個股漲勢稀落 指數一度大跌百點 台指期難拉高結算,2016-01-20,even
3192,Moneydj理財網,0,1.歐美股市走穩，不過受期指結算影響，台股週三開低，加上港股重挫，指數震盪走低，盤面上部份生...,1453281180692_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-20 16:16:00,重大要聞,yahoo股市,日盛投顧：台股短線參考區間7600-7900,2016-01-20,even


In [29]:
df_company2 = df_company[df_company['label5566'].str.contains('up|down|even')]
df_company2.head()

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,label5566
222,財訊快報,0,◆千億科技預算，張善政喊加碼，主計總處初步匡列1 061億、年增3%，可能再調高。<BR> ...,1451867210012_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 07:56:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-04,up
289,時報資訊,0,2016年開紅盤日，台股狂洩逾200點！儘管2015年最後一個交易日拉尾盤，守住8300點，...,1451879808470_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 11:34:00,重大要聞,yahoo股市,《盤中解析》紅盤日變臉，台股狂洩逾200點,2016-01-04,up
321,時報資訊,0,2016年紅盤日亞股倒光光，台股勉強守住8100點！大陸經濟數據不佳，人民幣續貶，且沙烏地阿...,1451891507662_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 14:45:00,重大要聞,yahoo股市,《盤後解析》陸股「熔斷」， 台股重挫險守8100點,2016-01-04,up
326,時報資訊,0,美股2015年封關日重挫，沙烏地阿拉伯宣布與伊朗斷交，中東情勢烏雲密布，今日亞股哀鴻遍野，台...,1451892413661_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 15:09:00,重大要聞,yahoo股市,《集中市場》亞股皮皮剉，三大法人同步賣超173.6億元,2016-01-04,up
1522,財訊快報,0,◆全台勞檢啟動，11類行業小心，其中284家電子零組件企業，實施全面普查。<BR> ◆海外分...,1452472319234_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:05:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-11,down


In [30]:
stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [31]:
content=""
for row in df_company2.index:
    try:
        content=content + " " + df_company2.loc[row,'content']
    except:
        continue

In [32]:
seg_generator = jieba.cut(content, cut_all=False)  # genarator
seglist = list(seg_generator)                     # 整篇文章string切出來的list
seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟

seglist[0:20]

Building prefix dict from /home/barry/projects/stock/dict.idkrsi.txt ...
Loading model from cache /tmp/jieba.ufc1c7fa02eb193cee2bb78b34d63c017.cache
Loading model cost 0.437 seconds.
Prefix dict has been built succesfully.


['千億',
 '科技',
 '預算',
 '張善政',
 '喊',
 '加碼',
 '主計',
 '總處',
 '初步',
 '匡列',
 '061',
 '億',
 '年增',
 '3%',
 '調高',
 '產創',
 '翻修',
 '教師',
 '持股',
 '上限']

In [33]:
wordcount = {}      # dictionary
for word in seglist:
    if str(word[0]).isdigit()==False:    # 切詞不是數字
        try:
            wordcount[word] = wordcount[word] + 1
        except:
            wordcount[word] = 1
            
list(wordcount.items())[0:20]

[('千億', 24),
 ('科技', 619),
 ('預算', 11),
 ('張善政', 5),
 ('喊', 61),
 ('加碼', 355),
 ('主計', 14),
 ('總處', 14),
 ('初步', 42),
 ('匡列', 1),
 ('億', 828),
 ('年增', 816),
 ('調高', 62),
 ('產創', 2),
 ('翻修', 3),
 ('教師', 3),
 ('持股', 415),
 ('上限', 35),
 ('擬', 96),
 ('鬆綁', 20)]

In [34]:
sorted_x = sorted(wordcount.items(), key=lambda kv: kv[1])  # sorted( dictionary.items() ) 也會變成list   # kv[1] 用第2個column排序
sorted_x.reverse()          # ascending 轉 descending
sorted_x[0:20]
sorted_x[:][:20]

[('點', 6236),
 ('台股', 6109),
 ('指數', 5582),
 ('億元', 5524),
 ('元', 5392),
 ('市場', 4053),
 ('被動', 3879),
 ('元件', 3864),
 ('股價', 3819),
 ('營收', 3542),
 ('外資', 3328),
 ('表現', 3195),
 ('國巨', 3077),
 ('持續', 2756),
 ('今年', 2508),
 ('族群', 2506),
 ('電子', 2385),
 ('中', 2290),
 ('公司', 2283),
 ('法人', 2166)]

In [35]:
len(sorted_x)

32306

In [36]:
df_features = pd.DataFrame(sorted_x, columns = ['word' , 'number'])                #,columns={"word","number"})
df_features.reset_index(drop=True)
df_features.head()

,word,number
0,點,6236
1,台股,6109
2,指數,5582
3,億元,5524
4,元,5392


In [37]:
df_features.to_csv("5pa_word.csv", index = False)

In [38]:
df_company2 = df_company[df_company['label5566'].str.contains('up')]
df_company2.head()

stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))
        
content=""
for row in df_company2.index:
    try:
        content=content + " " + df_company2.loc[row,'content']
    except:
        continue
        
seg_generator = jieba.cut(content, cut_all=False)  # genarator
seglist = list(seg_generator)                     # 整篇文章string切出來的list
seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟

seglist[0:20]

wordcount = {}      # dictionary
for word in seglist:
    if str(word[0]).isdigit()==False:    # 切詞不是數字
        try:
            wordcount[word] = wordcount[word] + 1
        except:
            wordcount[word] = 1
            
list(wordcount.items())[0:20]

sorted_x = sorted(wordcount.items(), key=lambda kv: kv[1])  # sorted( dictionary.items() ) 也會變成list   # kv[1] 用第2個column排序
sorted_x.reverse()          # ascending 轉 descending
sorted_x[0:20]
sorted_x[:][:20]

len(sorted_x)

df_features = pd.DataFrame(sorted_x, columns = ['word' , 'number'])                #,columns={"word","number"})
df_features.reset_index(drop=True)
df_features.head()

df_features.to_csv("5pa_word_up.csv", index = False)

In [39]:
df_company2 = df_company[df_company['label5566'].str.contains('down')]
df_company2.head()

stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))
        
content=""
for row in df_company2.index:
    try:
        content=content + " " + df_company2.loc[row,'content']
    except:
        continue
        
seg_generator = jieba.cut(content, cut_all=False)  # genarator
seglist = list(seg_generator)                     # 整篇文章string切出來的list
seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟

seglist[0:20]

wordcount = {}      # dictionary
for word in seglist:
    if str(word[0]).isdigit()==False:    # 切詞不是數字
        try:
            wordcount[word] = wordcount[word] + 1
        except:
            wordcount[word] = 1
            
list(wordcount.items())[0:20]

sorted_x = sorted(wordcount.items(), key=lambda kv: kv[1])  # sorted( dictionary.items() ) 也會變成list   # kv[1] 用第2個column排序
sorted_x.reverse()          # ascending 轉 descending
sorted_x[0:20]
sorted_x[:][:20]

len(sorted_x)

df_features = pd.DataFrame(sorted_x, columns = ['word' , 'number'])                #,columns={"word","number"})
df_features.reset_index(drop=True)
df_features.head()

df_features.to_csv("5pa_word_down.csv", index = False)

In [40]:
df_company2 = df_company[df_company['label5566'].str.contains('even')]
df_company2.head()

stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))
        
content=""
for row in df_company2.index:
    try:
        content=content + " " + df_company2.loc[row,'content']
    except:
        continue
        
seg_generator = jieba.cut(content, cut_all=False)  # genarator
seglist = list(seg_generator)                     # 整篇文章string切出來的list
seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟

seglist[0:20]

wordcount = {}      # dictionary
for word in seglist:
    if str(word[0]).isdigit()==False:    # 切詞不是數字
        try:
            wordcount[word] = wordcount[word] + 1
        except:
            wordcount[word] = 1
            
list(wordcount.items())[0:20]

sorted_x = sorted(wordcount.items(), key=lambda kv: kv[1])  # sorted( dictionary.items() ) 也會變成list   # kv[1] 用第2個column排序
sorted_x.reverse()          # ascending 轉 descending
sorted_x[0:20]
sorted_x[:][:20]

len(sorted_x)

df_features = pd.DataFrame(sorted_x, columns = ['word' , 'number'])                #,columns={"word","number"})
df_features.reset_index(drop=True)
df_features.head()

df_features.to_csv("5pa_word_even.csv", index = False)

In [41]:
df_company2 = df_company[df_company['label5566'].str.contains('up|down')]
df_company2.head()

stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))
        
content=""
for row in df_company2.index:
    try:
        content=content + " " + df_company2.loc[row,'content']
    except:
        continue
        
seg_generator = jieba.cut(content, cut_all=False)  # genarator
seglist = list(seg_generator)                     # 整篇文章string切出來的list
seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟

seglist[0:20]

wordcount = {}      # dictionary
for word in seglist:
    if str(word[0]).isdigit()==False:    # 切詞不是數字
        try:
            wordcount[word] = wordcount[word] + 1
        except:
            wordcount[word] = 1
            
list(wordcount.items())[0:20]

sorted_x = sorted(wordcount.items(), key=lambda kv: kv[1])  # sorted( dictionary.items() ) 也會變成list   # kv[1] 用第2個column排序
sorted_x.reverse()          # ascending 轉 descending
sorted_x[0:20]
sorted_x[:][:20]

len(sorted_x)

df_features = pd.DataFrame(sorted_x, columns = ['word' , 'number'])                #,columns={"word","number"})
df_features.reset_index(drop=True)
df_features.head()

df_features.to_csv("5pa_word_up_down.csv", index = False)